In [1]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [3]:
%cd '/content/drive/My Drive/resnet'
!ls

/content/drive/My Drive/resnet
 endings_predictor.ipynb   resized_images   results.xls
 finish.mp3		   resnet.ipynb    'Новая таблица.gsheet'


In [4]:
!pip install playsound
!pip install keras
!pip install tensorflow-gpu==2.0.0-alpha0

import tensorflow as tf
print(tf.__version__)

     |████████████████████████████████| 332.5MB 26kB/s 
     |████████████████████████████████| 51kB 5.7MB/s 
     |████████████████████████████████| 419kB 33.0MB/s 
     |████████████████████████████████| 3.0MB 39.3MB/s 


/usr/local/lib/python3.7/dist-packages/tensorflow/python/framework/dtypes.py:523: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.7/dist-packages/tensorflow/python/framework/dtypes.py:524: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/python3.7/dist-packages/tensorflow/python/framework/dtypes.py:525: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/lib/python3.7/dist-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or 

ImportError: ignored

In [5]:
# memory footprint support libraries/code
!ln -sf /opt/bin/nvidia-smi /usr/bin/nvidia-smi
!pip install gputil
!pip install psutil
!pip install humanize
import psutil
import humanize
import os
import GPUtil as GPU
GPUs = GPU.getGPUs()
# XXX: only one GPU on Colab and isn’t guaranteed
gpu = GPUs[0]
def printm():
 process = psutil.Process(os.getpid())
 print("Gen RAM Free: " + humanize.naturalsize( psutil.virtual_memory().available ), " | Proc size: " + humanize.naturalsize( process.memory_info().rss))
 print("GPU RAM Free: {0:.0f}MB | Used: {1:.0f}MB | Util {2:3.0f}% | Total {3:.0f}MB".format(gpu.memoryFree, gpu.memoryUsed, gpu.memoryUtil*100, gpu.memoryTotal))
printm() 

  Created wheel for gputil: filename=GPUtil-1.4.0-cp37-none-any.whl size=7411 sha256=e37b9b031c1dfae599d35033de1a1c1e2321710e572263a2f52efa221614c47c
  Stored in directory: /root/.cache/pip/wheels/3d/77/07/80562de4bb0786e5ea186911a2c831fdd0018bda69beab71fd
Successfully built gputil
Gen RAM Free: 12.7 GB  | Proc size: 289.0 MB
GPU RAM Free: 11441MB | Used: 0MB | Util   0% | Total 11441MB


In [ ]:
!kill -9 -1

In [6]:
import numpy as np
from tensorflow.keras.preprocessing.image import load_img, save_img, img_to_array, array_to_img
from IPython.display import Audio
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.applications.vgg16 import VGG16

from tensorflow.keras.callbacks import CSVLogger
from tensorflow.keras import optimizers
from tensorflow.keras.layers import Input, Add, Dense, Activation, ZeroPadding2D, BatchNormalization, Flatten, Conv2D, \
    AveragePooling2D, MaxPooling2D
from tensorflow.keras.models import Model, load_model
import tensorflow.keras.backend as K
from playsound import playsound
from xlwt import Workbook 

K.set_image_data_format('channels_last')
K.set_learning_phase(1)

ImportError: ignored

In [ ]:
def convert_to_one_hot(Y, C):
    Y = np.eye(C)[Y.reshape(-1)].T
    return Y
  

def load_dataset(dataset, num_classes=100, num_images = 100, batch_num=0):
    folders = ['A Polish Nobleman_Rembrandt',
               'A portrait of Dante_Giotto',
               'A Sunday Afternoon on the Island of La Grande Jatte_Georges Seurat',
               'Aline Marie Chazal Tristan_Paul Gauguin',
               'Amor Vincit Omnia_Caravaggio',
               'Annunciation_Jan van Eyck',
               'At the Races_Edgar Degas',
               'Avila Morning_Diego Rivera',
               'Basket of Fruit_Caravaggio',
               'Beethoven_Andy Warhol',
               
               'Benois Madonna_Leonardo da Vinci',
               'Calvary (Golgotha)_Marc Chagall',
               'Christ Before Pilate_Hieronymus Bosch',
               'Dance at Le Moulin de la Galette_Pierre-Auguste Renoir',
               'Demon Seated_Mikhail Vrubel',
               'Der Blaue Reiter_Wassily Kandinsky',
               'Dutch Interior_Salvador Dali',
               'Equestrian Portrait of Charles V_Titian',
               'Eugenia Primavesi_Gustav Klimt',
               'Evening; Red Tree_Piet Mondrian',
               
               'Femme assise_Pablo Picasso',
               'Fishermen at Sea_William Turner',
               'Flower Girl_Kazimir Malevich',
               'Flower Myth (Blumenmythos)_Paul Klee',
               'Four Breton Women_Paul Gauguin',
               'Greece on the Ruins of Missolonghi_Eugene Delacroix',
               'Horse Frightened by a Storm_Eugene Delacroix',
               'House with Palm Tree_Joan Miro',
               'Ignudo_Michelangelo',
               'Impression, Sunrise_Claude Monet',
               
               'In Bed_Henri de Toulouse-Lautrec',
               'Kiss of Judas_Giotto',
               'La toilette_Henri de Toulouse-Lautrec',
               'La Vie_Pablo Picasso',
               'Le Mur Rose_Henri Matisse',
               'Madonna and Child with St. John the Baptist_Sandro Botticelli',
               'Man in a Bowler Hat_Rene Magritte',
               'Memory (The Heart)_Frida Kahlo',
               'Modern Rome-Campo Vaccino_William Turner',
               'Molesey Weir-Morning_Alfred Sisley',
               
               'Munich-Schwabing with the Church of St. Ursula_Wassily Kandinsky',
               'Mural on Indian Red Ground_Jackson Pollock',
               'Music in the Tuileries_Edouard Manet',
               'No. 5_Jackson Pollock',
               'Old Woman and Boy with Candles_Peter Paul Rubens',
               'Old Woman Frying Eggs_Diego Velazquez',
               'Philip IV in Brown and Silver_Diego Velazquez',
               'Portrait of a Man in a Red Cap_Titian',
               'Portrait of Anna of Austria, Queen of France_Peter Paul Rubens',              
               'Portrait of Dr. Luigi Accame_Andy Warhol',
               
               'Portrait of Guidobaldo da Montefeltro_Raphael',
               'Portrait of Heriberto Casany_Joan Miro',
               'Portrait of Juan Gris_Amedeo Modigliani',
               'Portrait of Maude Abrantes_Amedeo Modigliani',
               'Portrait of Paul Cezanne_Camille Pissarro',
               'Portrait of the Artist Holding a Thistle_Albrecht Durer',
               'Praying hands_Albrecht Durer',
               'Red Balloon_Paul Klee',
               'Rest along the Stream. Edge of the Wood_Alfred Sisley',
               'Saint George and the Dragon_Raphael',
               
               'Saviour in Glory_Andrei Rublev',
               'Self-Portrait with Skeleton Arm_Edvard Munch',
               'Self-Portrait with Thorn Necklace and Hummingbird_Frida Kahlo',
               'Self-portrait, Man with a pipe_Gustave Courbet',
               'Starry Night Over the Rhone_Vincent van Gogh',
               'Still Life with a Curtain_Paul Cezanne',              
               'Still Life with Compote, Apples and Oranges_Henri Matisse',
               'Street in Avila_Diego Rivera',
               'Sunflowers_Vincent van Gogh',
               'The Arnolfini Portrait_Jan van Eyck',
               
               'The Bardi Altarpiece_Sandro Botticelli',
               'The Blind Leading the Blind_Pieter Bruegel',
               'The Blind Man_Gustav Klimt',
               'The Card Players_Paul Cezanne',
               'The Crucifixion of St Julia_Hieronymus Bosch',
               'The Crucifixion of St. Peter_Michelangelo',
               'The Dream_Henri Rousseau',
               'The Drunkard_Marc Chagall',
               'The Empire of Light_Rene Magritte',
               'The Last Supper_Leonardo da Vinci',
               
               'The Milkmaid of Bordeaux_Francisco Goya',
               'The Opening of the Fifth Seal_El Greco',
               'The Parasol_Francisco Goya',               
               'The Persistence of Memory_Salvador Dali',
               'The Procession to Calvary_Pieter Bruegel',
               'The Railway_Edouard Manet',
               'The Scream_Edvard Munch',
               'The Singer with the Glove_Edgar Degas',
               'The Storm on the Sea of Galilee_Rembrandt',
               'The Swan Princess_Mikhail Vrubel',
               
               'The Theater Box_Pierre-Auguste Renoir',
               'The Wave_Gustave Courbet',
               'Tiger in a Tropical Storm (Surprised!)_Henri Rousseau',
               'Trinity_Andrei Rublev',
               'Two Women Chatting by the Sea_Camille Pissarro',
               'View from the Dunes with Beach and Piers, Domburg_Piet Mondrian',
               'View of Toledo_El Greco',
               'Winter_Kazimir Malevich',
               'Woman with a Parasol, facing left_Claude Monet',
               'Young Woman Powdering Herself_Georges Seurat']

    fraction = int(num_images / 10)
    
    X = []
    
    if dataset == 'train':
      for item in folders:
        for i in range(batch_num * fraction, (batch_num + 1) * fraction):
          img = load_img('resized_images/' + item + '/image_' + str(i) + '.jpg')
          x = img_to_array(img)
          
          X.append(x)
          
      pattern = np.empty([1, fraction], dtype=int)
    elif dataset == 'test':
      for item in folders:
        for i in range(num_images - fraction * 3, num_images):
          img = load_img('resized_images/' + item + '/image_' + str(i) + '.jpg')
          x = img_to_array(img)
          
          X.append(x)
          
      pattern = np.empty([1, fraction * 3], dtype=int)

    X = np.stack(X, axis=0)

    # Normalize image vectors
    X /= 255

    Y = np.full_like(pattern, 0)

    for i in range(1, num_classes):
        Y = np.concatenate((Y, np.full_like(pattern, i)), axis=1)

    # Convert training or test labels to one hot matrices
    Y = convert_to_one_hot(Y, num_classes).T

    return X, Y

In [ ]:
X_test, Y_test = load_dataset(dataset = 'test')

Audio('finish.mp3', autoplay=True)

print("X_test shape: " + str(X_test.shape))
print("Y_test shape: " + str(Y_test.shape))

X_test shape: (3000, 224, 224, 3)
Y_test shape: (3000, 100)


In [ ]:
model = load_model('model_epoch1.h5')
print('Model loaded')

In [ ]:
def vgg16(classes=100):
  model = VGG16(include_top=False, weights=None)

  input=Input(shape=(224, 224, 3))

  X = model(input)

  X = AveragePooling2D((7, 7), name='avg_pool')(X)

  # output layer
  X = Flatten()(X)
  X = Dense(classes, activation='softmax', name='fc' + str(classes))(X)

  # Create model
  model = Model(inputs=input, outputs=X, name='VGG16')
  
  return model

model = vgg16()
print('Model created')

Model created


In [ ]:
model = VGG16(include_top=False, input_shape=(224, 224, 3), weights='imagenet')

model.summary()

In [ ]:
log_file = 'log.csv'
csv_logger = CSVLogger(log_file, append=True, separator=';')

In [ ]:
#optimizer = optimizers.SGD(lr=0.00001, momentum=0.9, nesterov=False)
#adabound = AdaBound(lr=0.0001, final_lr=0.1)
optimizer = optimizers.Adam(lr=0.000001, beta_1=0.9, beta_2=0.999, epsilon=1e-8, decay=0.0, amsgrad=False)
model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
wb = Workbook()
sheet = wb.add_sheet('Sheet')
sheet.write(0, 0, 'Test Error')

In [ ]:
def main(show_mistakes=False):
    #model.summary()
    
    for i in range(9, 11):
      for j in range(7):
        X_train_batch, Y_train_batch = load_dataset(dataset = 'train', batch_num = j)
        model.fit(X_train_batch, Y_train_batch, batch_size=128, epochs=i + 1, verbose=2, initial_epoch=i)
        
      predictions = model.evaluate(X_test, Y_test, verbose=2)
      sheet.write(i + 1, 0, 1 - predictions[1])
      
        # print("Loss = " + str(predictions[0]))
        #print("Test Error = " + str(1 - predictions[1]))

In [ ]:
main()
#Audio('finish.mp3', autoplay=True)

Epoch 10/10
1000/1000 - 15s - loss: 0.0095 - accuracy: 0.9980
Epoch 10/10
1000/1000 - 15s - loss: 0.0047 - accuracy: 0.9980
Epoch 10/10
1000/1000 - 16s - loss: 0.0035 - accuracy: 0.9980
Epoch 10/10
1000/1000 - 16s - loss: 0.0031 - accuracy: 0.9990
Epoch 10/10
1000/1000 - 15s - loss: 0.0012 - accuracy: 1.0000
Epoch 10/10
1000/1000 - 15s - loss: 0.0032 - accuracy: 0.9990
Epoch 10/10
1000/1000 - 16s - loss: 0.0035 - accuracy: 0.9990
3000/3000 - 15s - loss: 0.0194 - accuracy: 0.9960
Epoch 11/11
1000/1000 - 15s - loss: 0.0023 - accuracy: 1.0000
Epoch 11/11
1000/1000 - 15s - loss: 0.0033 - accuracy: 1.0000
Epoch 11/11
1000/1000 - 16s - loss: 0.0016 - accuracy: 1.0000
Epoch 11/11
1000/1000 - 16s - loss: 0.0025 - accuracy: 1.0000
Epoch 11/11
1000/1000 - 15s - loss: 0.0012 - accuracy: 1.0000
Epoch 11/11
1000/1000 - 16s - loss: 0.0030 - accuracy: 0.9990
Epoch 11/11
1000/1000 - 15s - loss: 0.0032 - accuracy: 0.9990
3000/3000 - 15s - loss: 0.0189 - accuracy: 0.9960


In [ ]:
wb.save('results.xls')

In [ ]:
model.save('Model.h5')
print('Model saved')

In [ ]:
P = np.round(model.predict(X_test))

for i in range(Y_test.shape[0]):
  for j in range(Y_test.shape[1]):
    if Y_test[i, j] != P[i, j]:
      print('Mislabeled image index: ', i)
      print('Expected values: ', Y_test[i])
      print('Predicted values: ', P[i])
      break

In [ ]:
print(np.round(model.predict(X_test[1122:1130])))